# AlexNet (Transfer Learning)

In [26]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchsummary import summary
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from PIL import Image
from ptflops import get_model_complexity_info
import time

In [27]:
# Device configuration (Select CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n Device : {device}")


 Device : cuda


### Load and prepare the data

In [28]:
# Transformations for the dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale-like images to 3 channels
    transforms.Resize((256, 256)),               # Resize images for AlexNet input
    transforms.CenterCrop(227),                  # Crop the central 227×227 region
    transforms.ToTensor(),                       # Convert images to tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

# Custom dataset class to handle CSV and image folder
class SARImageDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = f"{self.image_folder}/{self.data.iloc[idx, 0]}.png"
        label = self.data.iloc[idx, 1]
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image, label

# Load full training dataset
train_val_dataset = SARImageDataset(csv_file="data/train.csv", image_folder="data/images_train", transform=transform)
test_dataset = SARImageDataset(csv_file="data/test.csv", image_folder="data/images_test", transform=transform)

# Split into training and validation datasets
train_size = int(0.8 * len(train_val_dataset))  # 80% for training
val_size = len(train_val_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Load pre-trained ResNet50 model

In [29]:
# Load pre-trained AlexNet
model = models.alexnet(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.classifier[6].in_features  # Get the input features of the last layer
model.classifier[6] = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print summary
summary(model, input_size=(3, 227, 227))

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

### Train the model

In [30]:
# Initialize a DataFrame to log training/validation metrics
log_df = pd.DataFrame(columns=["Epoch", "Train Loss", "Validation Loss", "Validation Accuracy"])

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    
    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, (3, 224, 224), as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"FLOPs: {(flops / 1e9 ):.2f} GFLOPs")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "model_outputs_data/best_alexnet_model.pth")


# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df.to_csv("model_outputs_data/model_evaluation_logs/training_logs_alexnet.csv", index=False)

C:\Users\ishar\AppData\Local\Temp\ipykernel_58276\1540578241.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)


Epoch [1/50], Train Loss: 0.8037, Val Loss: 0.6840, Val Accuracy: 52.42%, FLOPs: 0.71 GFLOPs
Epoch [2/50], Train Loss: 0.6883, Val Loss: 0.6898, Val Accuracy: 53.16%, FLOPs: 0.71 GFLOPs
Epoch [3/50], Train Loss: 0.6796, Val Loss: 0.7074, Val Accuracy: 49.49%, FLOPs: 0.71 GFLOPs
Epoch [4/50], Train Loss: 0.6731, Val Loss: 0.6427, Val Accuracy: 63.88%, FLOPs: 0.71 GFLOPs
Epoch [5/50], Train Loss: 0.6559, Val Loss: 0.6917, Val Accuracy: 49.49%, FLOPs: 0.71 GFLOPs
Epoch [6/50], Train Loss: 0.6244, Val Loss: 0.6641, Val Accuracy: 74.16%, FLOPs: 0.71 GFLOPs
Epoch [7/50], Train Loss: 0.5444, Val Loss: 0.6641, Val Accuracy: 75.77%, FLOPs: 0.71 GFLOPs
Epoch [8/50], Train Loss: 0.4839, Val Loss: 0.4248, Val Accuracy: 82.09%, FLOPs: 0.71 GFLOPs
Epoch [9/50], Train Loss: 0.4515, Val Loss: 0.4380, Val Accuracy: 78.41%, FLOPs: 0.71 GFLOPs
Epoch [10/50], Train Loss: 0.4435, Val Loss: 0.4835, Val Accuracy: 75.92%, FLOPs: 0.71 GFLOPs
Epoch [11/50], Train Loss: 0.4361, Val Loss: 0.4502, Val Accuracy: 82

### Prediction and Model evaluation

In [31]:
# Test the model
# model.load_state_dict(torch.load("model_outputs_data/best_alexnet_model.pth"))  # Load the best model for testing
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute Evaluation Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["No Internal Waves", "Internal Waves"]))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

C:\Users\ishar\AppData\Local\Temp\ipykernel_58276\126773375.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_outputs_data/best_ale


Test Accuracy: 86.43%
Precision: 0.92
Recall: 0.81
F1-Score: 0.86

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.82      0.92      0.87       719
   Internal Waves       0.92      0.81      0.86       740

         accuracy                           0.86      1459
        macro avg       0.87      0.87      0.86      1459
     weighted avg       0.87      0.86      0.86      1459


Confusion Matrix:
[[664  55]
 [143 597]]
